In [1]:
import re
import subprocess
from typing import List

import pandas as pd
import glob
from pathlib import Path
from tqdm import tqdm

In [2]:
DATA_PATH = Path("../data")
DATA_PATH.mkdir(exist_ok=True)

In [3]:
download_command = [
    "aws", "s3", "sync",
    "--no-sign-request",
    "s3://physionet-open/challenge-2019/1.0.0/training/",
    str(DATA_PATH)
]

process = subprocess.Popen(
    download_command,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1
)

for line in process.stdout:
    print(line, end="")

return_code = process.wait()
if return_code != 0:
    raise subprocess.CalledProcessError(return_code, download_command)

Completed 2.9 KiB/~6.2 MiB (808 Bytes/s) with ~1005 file(s) remaining (calculating...)
download: s3://physionet-open/challenge-2019/1.0.0/training/training_setA/p000006.psv to ..\data\training_setA\p000006.psv
Completed 2.9 KiB/~6.2 MiB (808 Bytes/s) with ~1004 file(s) remaining (calculating...)
Completed 11.8 KiB/~6.2 MiB (3.2 KiB/s) with ~1005 file(s) remaining (calculating...) 
download: s3://physionet-open/challenge-2019/1.0.0/training/training_setA/p000001.psv to ..\data\training_setA\p000001.psv
Completed 11.8 KiB/~6.2 MiB (3.2 KiB/s) with ~1004 file(s) remaining (calculating...)
Completed 15.5 KiB/~6.2 MiB (4.2 KiB/s) with ~1005 file(s) remaining (calculating...)
Completed 23.3 KiB/~6.2 MiB (6.3 KiB/s) with ~1005 file(s) remaining (calculating...)
download: s3://physionet-open/challenge-2019/1.0.0/training/training_setA/p000010.psv to ..\data\training_setA\p000010.psv
Completed 23.3 KiB/~6.2 MiB (6.3 KiB/s) with ~1004 file(s) remaining (calculating...)
download: s3://physionet-o

KeyboardInterrupt: 

In [4]:
patient_file_paths = glob.glob(str(DATA_PATH / "training_set*/*.psv"))

In [10]:
dfs: List[pd.DataFrame] = []
for patient_file in tqdm(patient_file_paths, desc="Loading .psv files"):
    df = pd.read_csv(patient_file, sep='|')

    match = re.search(r'p(\d+)\.psv$', patient_file)
    if not match:
        continue

    patient_id = int(match.group(1)) if match else None
    df["Patient_ID"] = patient_id
    dfs.append(df)

combined_df = pd.concat(dfs, ignore_index=True)

Loading .psv files: 100%|██████████| 4403/4403 [00:06<00:00, 639.77it/s]


In [14]:
OUTPUT_PATH = DATA_PATH / "dataset.parquet"
combined_df.to_parquet(OUTPUT_PATH, engine="pyarrow")
print(f"Saved combined dataset to {OUTPUT_PATH}")

Saved combined dataset to ..\data\dataset.parquet
